# Trying to upload DICOMs and ROIs (DICOM-RTSTRUCTs) to XNAT using XNATpy

### https://xnat.readthedocs.io/en/latest/index.html

### Changes from v1:  
#### - Upload all DICOMs whose UIDs correspond to the UIDs of the ROI data.  
#### - May need to modify the Project/Subject/Experiment(Session) labels in the DICOM data and RTSTRUCT file to match the labels assigned when creating new Project/Subject/Experiment...

In [1]:
import xnat
#xnat.__version__ # 0.3.21
print(f'XNATpy version = {xnat.__version__}') # 0.3.21
import SimpleITK as sitk
import pydicom
import dicom2nifti
import numpy as np
#import math
import matplotlib.pyplot as plt
#from matplotlib import pyplot
import matplotlib.image as mpimg
import os, sys, time
import natsort
import dicom_functions
import explore_dicom_data
from myshow import myshow
import importlib

XNATpy version = 0.3.21


#### Connect to XNAT server:

In [2]:
xnatAddress = 'http://10.1.1.17'
#xnatAddress = 'http://10.1.1.17/app/template/Index.vm'
#xnatAddress = 'http://10.1.1.17/app/template/Login.vm#!'
#xnatAddress = 'http://10.1.1.17:8080/xnat'
#xnatAddress = 'http://10.1.1.17:80/xnat'
#xnatAddress = 'http://localhost:8080/xnat'
#xnatAddress = 'http://10.1.1.17:8080/admin'
#xnatAddress = 'http://localhost:8080/admin'

#session = xnat.connect(xnatAddress, user='xnat', password='xnat')
#session = xnat.connect(xnatAddress, user='owner', password='owner')
session = xnat.connect(xnatAddress, user='admin', password='admin', debug=True)

[2020-02-18 15:52:24,969] DEBUG: __init__:197 >> Response url: http://10.1.1.17/data/projects
[2020-02-18 15:52:24,969] DEBUG: __init__:59 >> Getting http://10.1.1.17/ to test auth
[2020-02-18 15:52:25,228] DEBUG: __init__:61 >> Status: 200
[2020-02-18 15:52:25,229] INFO: __init__:95 >> Logged in successfully as admin
[2020-02-18 15:52:25,232] DEBUG: session:334 >> GET URI http://10.1.1.17/data/JSESSION
[2020-02-18 15:52:25,306] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 15:52:25,308] DEBUG: session:334 >> GET URI http://10.1.1.17/data/version
[2020-02-18 15:52:25,325] DEBUG: session:307 >> Received response with status code: 404
[2020-02-18 15:52:25,326] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/siteConfig/buildInfo?format=json
[2020-02-18 15:52:25,331] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 15:52:25,332] INFO: __init__:239 >> Found an 1.7 version (1.7.5.6)
[2020-02-18 15:52:25,332] DEBUG: session:334 >> GET U

[2020-02-18 15:52:25,470] DEBUG: convert_xsd:1237 >> Adding EEGSession -> xnat:eegSessionData to class name map
[2020-02-18 15:52:25,471] DEBUG: convert_xsd:1237 >> Adding OtherDicomSession -> xnat:otherDicomSessionData to class name map
[2020-02-18 15:52:25,471] DEBUG: convert_xsd:1237 >> Adding SubjectVariables -> xnat:subjectVariablesData to class name map
[2020-02-18 15:52:25,472] DEBUG: convert_xsd:1237 >> Adding PETSession -> xnat:petSessionData to class name map
[2020-02-18 15:52:25,472] DEBUG: convert_xsd:1237 >> Adding PETMRSession -> xnat:petmrSessionData to class name map
[2020-02-18 15:52:25,472] DEBUG: convert_xsd:1237 >> Adding MRScan -> xnat:mrScanData to class name map
[2020-02-18 15:52:25,474] DEBUG: convert_xsd:1237 >> Adding CTScan -> xnat:ctScanData to class name map
[2020-02-18 15:52:25,476] DEBUG: convert_xsd:1237 >> Adding PETScan -> xnat:petScanData to class name map
[2020-02-18 15:52:25,477] DEBUG: convert_xsd:1237 >> Adding EPSScan -> xnat:epsScanData to class

[2020-02-18 15:52:25,530] DEBUG: convert_xsd:932 >> Found 3 unknown tags: {'{http://nrg.wustl.edu/xdat}relation', '{http://nrg.wustl.edu/security}element', '{http://nrg.wustl.edu/security}field'}
[2020-02-18 15:52:25,531] INFO: convert_xsd:949 >> === Retrieving schema from /xapi/schemas/assessments ===
[2020-02-18 15:52:25,532] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/schemas/assessments
[2020-02-18 15:52:25,561] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 15:52:25,563] DEBUG: convert_xsd:919 >> Registering namespace: xs  ->  http://www.w3.org/2001/XMLSchema
[2020-02-18 15:52:25,564] DEBUG: convert_xsd:919 >> Registering namespace: xnat  ->  http://nrg.wustl.edu/xnat
[2020-02-18 15:52:25,565] DEBUG: convert_xsd:919 >> Registering namespace: xnat_a  ->  http://nrg.wustl.edu/xnat_assessments
[2020-02-18 15:52:25,566] DEBUG: convert_xsd:1241 >> Skipping non-class top-level tag {http://www.w3.org/2001/XMLSchema}import
[2020-02-18 15:52:25,568] DEBUG

[2020-02-18 15:52:25,691] DEBUG: convert_xsd:919 >> Registering namespace: arc  ->  http://nrg.wustl.edu/arc
[2020-02-18 15:52:25,691] DEBUG: convert_xsd:919 >> Registering namespace: xs  ->  http://www.w3.org/2001/XMLSchema
[2020-02-18 15:52:25,691] DEBUG: convert_xsd:919 >> Registering namespace: xdat  ->  http://nrg.wustl.edu/xdat
[2020-02-18 15:52:25,691] DEBUG: convert_xsd:1237 >> Adding ArchiveSpecification -> None to class name map
[2020-02-18 15:52:25,693] DEBUG: convert_xsd:932 >> Found 3 unknown tags: {'{http://nrg.wustl.edu/xdat}relation', '{http://nrg.wustl.edu/security}element', '{http://nrg.wustl.edu/security}field'}
[2020-02-18 15:52:25,693] INFO: convert_xsd:949 >> === Retrieving schema from /xapi/schemas/validation/protocolValidation ===
[2020-02-18 15:52:25,694] DEBUG: session:334 >> GET URI http://10.1.1.17/xapi/schemas/validation/protocolValidation
[2020-02-18 15:52:25,708] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 15:52:25,710] DEBUG

[2020-02-18 15:52:25,793] INFO: convert_xsd:1356 >> Removing class xnatpy:roiCollectionDataReferences from parser, it only has one element!
[2020-02-18 15:52:25,793] INFO: convert_xsd:1356 >> Removing class xnatpy:role_typeAllowed_actions from parser, it only has one element!
[2020-02-18 15:52:25,794] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityPrimary_security_fields from parser, it only has one element!
[2020-02-18 15:52:25,795] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityElement_actions from parser, it only has one element!
[2020-02-18 15:52:25,795] INFO: convert_xsd:1356 >> Removing class xnatpy:element_securityListing_actions from parser, it only has one element!
[2020-02-18 15:52:25,796] DEBUG: convert_xsd:1362 >> Ignoring non-listing...
[2020-02-18 15:52:25,797] DEBUG: convert_xsd:1363 >> Element class: {'parser': <xnat.convert_xsd.SchemaParser object at 0x0000023D2941D6C8>, 'name': 'xnatpy:userPrimary_password', 'logger': <Logger xnat-ce9

...

[2020-02-18 15:52:25,969] INFO: convert_xsd:1010 >> Processing xnat:ioScanData (base class xnat:imageScanData)
[2020-02-18 15:52:25,969] INFO: convert_xsd:1010 >> Processing xnat:usSessionData (base class xnat:imageSessionData)
[2020-02-18 15:52:25,969] INFO: convert_xsd:1010 >> Processing xnat:usScanData (base class xnat:imageScanData)
[2020-02-18 15:52:25,970] INFO: convert_xsd:1010 >> Processing xnat:ecgSessionData (base class xnat:imageSessionData)
[2020-02-18 15:52:25,970] INFO: convert_xsd:1010 >> Processing xnat:ecgScanData (base class xnat:imageScanData)
[2020-02-18 15:52:25,970] INFO: convert_xsd:1010 >> Processing xnat:hdSessionData (base class xnat:imageSessionData)
[2020-02-18 15:52:25,971] INFO: convert_xsd:1010 >> Processing xnat:hdScanData (base class xnat:imageScanData)
[2020-02-18 15:52:25,971] INFO: convert_xsd:1010 >> Processing xnat:epsSessionData (base class xnat:imageSessionData)
[2020-02-18 15:52:25,971] INFO: convert_xsd:1010 >> Processing xnat:epsScanData (base

[2020-02-18 15:52:26,003] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataParametersDatarecord (base class None)
[2020-02-18 15:52:26,004] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataChannels (base class None)
[2020-02-18 15:52:26,004] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataSoftwarefiltersimpedances (base class None)
[2020-02-18 15:52:26,005] INFO: convert_xsd:1010 >> Processing xnatpy:eegScanDataSoftwarefiltersimpedancesImpedance (base class None)
[2020-02-18 15:52:26,005] INFO: convert_xsd:1010 >> Processing xnat_a:sideEffectsPittsburghData (base class xnat:subjectAssessorData)
[2020-02-18 15:52:26,007] INFO: convert_xsd:1010 >> Processing xnat_a:ybocsData (base class xnat:subjectAssessorData)
[2020-02-18 15:52:26,009] INFO: convert_xsd:1010 >> Processing xnat_a:ygtssData (base class xnat:subjectAssessorData)
[2020-02-18 15:52:26,010] INFO: convert_xsd:1010 >> Processing xnatpy:ygtssDataMotor (base class None)
[2020-02-18 15:52:26,011] INFO: convert_

[2020-02-18 15:52:26,063] INFO: convert_xsd:1010 >> Processing arc:pipelineParameterData (base class None)
[2020-02-18 15:52:26,064] DEBUG: convert_xsd:997 >> Wait with processing xnatpy:pipelineParameterDataCsvvalues because base xnatpy:csvValuesString is not yet processed
[2020-02-18 15:52:26,064] INFO: convert_xsd:1010 >> Processing xnatpy:csvValuesString (base class None)
[2020-02-18 15:52:26,064] INFO: convert_xsd:1010 >> Processing arc:pathInfo (base class None)
[2020-02-18 15:52:26,065] DEBUG: convert_xsd:997 >> Wait with processing arc:property because base xnatpy:propertyString is not yet processed
[2020-02-18 15:52:26,065] INFO: convert_xsd:1010 >> Processing xnatpy:propertyString (base class None)
[2020-02-18 15:52:26,066] DEBUG: convert_xsd:997 >> Wait with processing arc:fieldSpecification because base xnatpy:fieldSpecificationString is not yet processed
[2020-02-18 15:52:26,066] INFO: convert_xsd:1010 >> Processing xnatpy:fieldSpecificationString (base class None)
[2020-0

[2020-02-18 15:52:26,098] INFO: convert_xsd:1010 >> Processing xnat:petAssessorData (base class xnat:imageAssessorData)
[2020-02-18 15:52:26,100] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerDose (base class xnatpy:doseFloat)
[2020-02-18 15:52:26,101] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerTotalmass (base class xnatpy:totalMassFloat)
[2020-02-18 15:52:26,102] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerIntermediate (base class xnatpy:intermediateFloat)
[2020-02-18 15:52:26,104] INFO: convert_xsd:1010 >> Processing xnatpy:petSessionDataTracerIsotope (base class xnatpy:isotopeString)
[2020-02-18 15:52:26,105] INFO: convert_xsd:1010 >> Processing xnat:petScanData (base class xnat:imageScanData)
[2020-02-18 15:52:26,107] INFO: convert_xsd:1010 >> Processing xnatpy:petScanDataParameters (base class None)
[2020-02-18 15:52:26,111] INFO: convert_xsd:1010 >> Processing xnatpy:petScanDataParametersFrames (base class None)
[2020-02-

[2020-02-18 15:52:26,150] INFO: convert_xsd:1021 >> Missed: []
[2020-02-18 15:52:26,151] INFO: convert_xsd:1022 >> Missed base class: []
[2020-02-18 15:52:26,151] DEBUG: convert_xsd:1023 >> Spent 2 iterations
[2020-02-18 15:52:26,164] DEBUG: __init__:249 >> Code file written to: C:\Users\ctorti\AppData\Local\Temp\tmp9v67hbzn_generated_xnat.py
[2020-02-18 15:52:26,367] DEBUG: __init__:256 >> Loaded generated module


In [4]:
session

#### Define the REST path variables:

In [3]:
# Define new project and subject labels, and session number:
projectLabel = 'BrainTumorProg_copy'
subjectLabel = 'PGM-002_Feb18'
experimentLabel = subjectLabel + '_MR_1'
#experimentNo = 0 # (= 1 in XNAT web app; e.g. ‘PGM-002_MR_1’)
#scanNo = 0 # 0 -> T1post
print('Project label =', projectLabel)
print('Subject label =', subjectLabel)
print('Experiment label =', experimentLabel)

Project label = BrainTumorProg_copy
Subject label = PGM-002_Feb18
Experiment label = PGM-002_Feb18_MR_1


#### Location of previously downloaded DICOM and DICOM-RTSTRUCT files:

In [4]:
# The DICOM files were exported here:
dicomDLdir = r'C:\Code\WP1.3_multiple_modalities\trying stuff\XNAT downloads\BrainTumorProg\PGM-002\0\0\PGM-002_MR_1\scans\11-T1post\resources\DICOM\files'

# And the ROI DICOM-RTStruct file was exported here:
roiDLdir = r'C:\Code\WP1.3_multiple_modalities\trying stuff\XNAT downloads\BrainTumorProg\PGM-002\0\0\AIM_20200123_145716\out\resources\RTSTRUCT\files'

#### Create a new project in XNAT:

In [5]:
project = session.classes.ProjectData(parent=session, name=projectLabel)

[2020-02-18 15:54:02,485] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy
[2020-02-18 15:54:02,486] DEBUG: core:180 >> query: {'xsiType': 'xnat:projectData', 'req_format': 'qs', 'name': 'BrainTumorProg_copy'}
[2020-02-18 15:54:02,487] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy?xsiType=xnat%3AprojectData&req_format=qs&name=BrainTumorProg_copy
[2020-02-18 15:54:02,487] DEBUG: session:430 >> PUT DATA None
[2020-02-18 15:54:02,488] DEBUG: session:431 >> PUT FILES None
[2020-02-18 15:54:02,573] DEBUG: session:307 >> Received response with status code: 200


#### Create a new subject for the new project:

In [57]:
subject = session.classes.SubjectData(parent=project, label=subjectLabel)

[2020-02-19 15:52:27,189] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb18
[2020-02-19 15:52:27,190] DEBUG: core:180 >> query: {'xsiType': 'xnat:subjectData', 'req_format': 'qs', 'label': 'PGM-002_Feb18'}
[2020-02-19 15:52:27,191] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb18?xsiType=xnat%3AsubjectData&req_format=qs&label=PGM-002_Feb18
[2020-02-19 15:52:27,192] DEBUG: session:430 >> PUT DATA None
[2020-02-19 15:52:27,193] DEBUG: session:431 >> PUT FILES None
[2020-02-19 15:52:27,230] DEBUG: session:307 >> Received response with status code: 200


#### Create a new experiment for the new subject:

##### Note:  Spaces are not permitted for labels!

In [7]:
experiment = session.classes.MrSessionData(parent=subject, label=experimentLabel)

[2020-02-18 15:54:10,251] DEBUG: session:334 >> GET URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb18?format=json
[2020-02-18 15:54:10,338] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 15:54:10,339] DEBUG: core:171 >> PUT URI: /data/archive/projects/BrainTumorProg_copy/subjects/XNAT_S00009/experiments/PGM-002_Feb18_MR_1
[2020-02-18 15:54:10,339] DEBUG: core:180 >> query: {'xsiType': 'xnat:mrSessionData', 'req_format': 'qs', 'label': 'PGM-002_Feb18_MR_1'}
[2020-02-18 15:54:10,340] DEBUG: session:428 >> PUT URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/XNAT_S00009/experiments/PGM-002_Feb18_MR_1?xsiType=xnat%3AmrSessionData&req_format=qs&label=PGM-002_Feb18_MR_1
[2020-02-18 15:54:10,342] DEBUG: session:430 >> PUT DATA None
[2020-02-18 15:54:10,343] DEBUG: session:431 >> PUT FILES None
[2020-02-18 15:54:10,539] DEBUG: session:307 >> Received response with status code: 201


#### Load the RTSTRUCT file to the get the Referenced SOP Instance UIDs

In [28]:
# Get the filename of the DICOM-RTSTRUCT file:
roiFname = os.listdir(roiDLdir)
roiFname = roiFname[0]

roiFpath = os.path.join(roiDLdir, roiFname)

# Load the DICOM-RTStruct file:
roi = pydicom.dcmread(roiFpath)

roi

(0008, 0016) SOP Class UID                       UI: RT Structure Set Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.826.0.1.534147.578.4208788039.20200231457312
(0008, 0020) Study Date                          DA: '19960813'
(0008, 0030) Study Time                          TM: '162908'
(0008, 0050) Accession Number                    SH: ''
(0008, 0060) Modality                            CS: 'RTSTRUCT'
(0008, 0070) Manufacturer                        LO: 'GE MEDICAL SYSTEMS'
(0008, 0090) Referring Physician's Name          PN: ''
(0008, 1070) Operators' Name                     PN: ''
(0008, 1090) Manufacturer's Model Name           LO: 'Optima MR450w'
(0010, 0010) Patient's Name                      PN: 'PGM-002'
(0010, 0020) Patient ID                          LO: 'PGM-002_MR_1'
(0010, 0030) Patient's Birth Date                DA: '00021130'
(0010, 0040) Patient's Sex                       CS: 'M'
(0018, 1020) Software Version(s)                 LO: ['25', 'LX', '

#### Get a list of the Referenced SOP Instance UIDs in roi

In [30]:
roiUids = []

# Loop for each contour sequence:
for sequence in roi.ROIContourSequence[0].ContourSequence:
    roiUids.append(sequence.ContourImageSequence[0].ReferencedSOPInstanceUID)
    
roiUids

['1.3.6.1.4.1.14519.5.2.1.4429.7055.320280293613596547840725755465',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.677643793103545804182040061154',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.149742284257271356119234155647',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.207269894846632100106605315699',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.207269894846632100106605315699',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.337321577215686178552541657021',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.337321577215686178552541657021',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.121761661581454976960293845378',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.176309996049760286843247671656',
 '1.3.6.1.4.1.14519.5.2.1.4429.7055.213859255991588180723164022910',
 '1.3.6.1.4.1.14519.5.2.1.4429.705

#### Read in each DICOM and create a list of indeces that correspond to the DICOMs that have SOP Instance UIDs that match the Reference SOP Instance UIDs in the ROI data

##### Note:  The ordering of inds is not the same as the ordering of the frame number since the list of DICOM files are not read in using natsort!

In [34]:
# Get the filenames of the DICOM files:
dicomFnames = os.listdir(dicomDLdir)
#dicomFnames

# Create a list of full file paths:
dicomFpaths = []

# Initialise array of indices:
inds = []

# Loop for each file name in dicomFnames:
for f in range(len(dicomFnames)):
    dicomFpath = os.path.join(dicomDLdir, dicomFnames[f])
    
    dicomFpaths.append(dicomFpath)
    
    # Read in the dicom file:
    dicom = pydicom.dcmread(dicomFpath)
    
    dicomUid = dicom.SOPInstanceUID
    
    # If dicomUid is in roiUids append f in inds:
    if dicomUid in roiUids:
        inds.append(f)
        
inds

[3, 4, 5, 6, 7, 8, 9, 10, 12]

In [14]:
# Get a list of sessions waiting for archiving:

session.prearchive.sessions()

[2020-02-18 16:34:29,225] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-18 16:34:29,253] DEBUG: session:307 >> Received response with status code: 200


[<PrearchiveSession BrainTumorProg_copy/20200217_144817230/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200206_152623716/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200207_144303310/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101919731/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_213423578/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101920246/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_204058544/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101918136/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_215434501/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_161330203/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_162336865/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200214_085602810/PGM-002_Feb14_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101916509/PGM-002_Feb17_MR_1>,
 <

## Instead of selecting only the DICOMs that have UIDs within the ROI data simply upload them all:

In [8]:
# Get the full filepaths of the DICOM files using the helper function (dc_filepaths(),
# which uses natsort for creating the list with natural sorting):
dicomFpaths = dicom_functions.dc_filepaths(dicomDLdir)

#### Try uploading data to XNAT

##### Note:  It should be possible to upload an entire directory of files but I'm not sure how, and for the benefit of working out what conditions need to be met to successfully upload an RTSTRUCT file, I'll only upload the DICOMs that have corresponding ROI contour data.

In [29]:
# Upload to the prearchive all DICOMs:
for f in range(len(dicomFpaths)):
    print(f'Uploading file {f+1} of {len(dicomFpaths)}...')
    
    prearchive_session = session.services.import_(dicomFpaths[f], \
                                                  project=projectLabel, \
                                                  subject=subjectLabel, \
                                                  experiment=experimentLabel, \
                                                  destination='/prearchive')

[2020-02-18 17:03:53,921] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1


Uploading file 1 of 22...


[2020-02-18 17:03:54,447] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:03:54,448] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1


Uploading file 2 of 22...


[2020-02-18 17:03:54,969] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:03:54,971] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1


...

[2020-02-18 17:04:05,265] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:04:05,266] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1


Uploading file 21 of 22...


[2020-02-18 17:04:05,761] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:04:05,763] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1


Uploading file 22 of 22...


[2020-02-18 17:04:06,261] DEBUG: session:307 >> Received response with status code: 200


In [16]:
# Get a list of sessions waiting for archiving:

session.prearchive.sessions()

[2020-02-18 16:36:19,915] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-18 16:36:19,952] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 16:36:19,953] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_163529257/PGM_002_MR_1?format=json
[2020-02-18 16:36:19,960] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 16:36:19,961] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_163533097/PGM_002_MR_1?format=json
[2020-02-18 16:36:19,972] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 16:36:19,973] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_163531403/PGM_002_MR_1?format=json
[2020-02-18 16:36:19,981] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 16:36:19,982] DEBUG: session:334 >> GET 

[<PrearchiveSession BrainTumorProg_copy/20200218_163529257/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_144817230/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163533097/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200206_152623716/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200207_144303310/PGM_002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163531403/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163529798/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101919731/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163528162/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_213423578/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101920246/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200212_204058544/PGM-002_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_101918136/PGM-0

In [17]:
session.prearchive.find(status='READY')

[2020-02-18 16:36:35,229] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-18 16:36:35,243] DEBUG: session:307 >> Received response with status code: 200


[<PrearchiveSession BrainTumorProg_copy/20200218_163529257/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200217_144817230/PGM-002_Feb17_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163533097/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163531403/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163529798/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163528162/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163528708/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200214_085602810/PGM-002_Feb14_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163530868/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163530326/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163532508/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163531934/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/

In [20]:
# Get a list of the files in the prearchive that have READY status 
# and the session label experimentLabel:
session.prearchive.find(status='READY', session=experimentLabel)

[2020-02-18 16:40:48,035] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-18 16:40:48,059] DEBUG: session:307 >> Received response with status code: 200


[<PrearchiveSession BrainTumorProg_copy/20200218_163529257/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163533097/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163531403/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163529798/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163528162/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163528708/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163530868/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163530326/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163532508/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163531934/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163533725/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_163534273/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/

In [21]:
# Delete select files from the prearchive:
# This could be useful but don't know how to delete by Upload date...

#session.prearchive.PrearchiveSession.uploaded()

In [22]:
#session.prearchive.find(project=projectLabel, subject=subjectLabel, session=experimentLabel, status='READY')

In [31]:
# Create a session of the files within the prearchive that I want to archive:
prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                        subject=subjectLabel, \
                                                        session=experimentLabel, \
                                                        status='READY')

[2020-02-18 17:07:21,078] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-18 17:07:21,106] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:21,108] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_170354463/PGM_002_MR_1?format=json
[2020-02-18 17:07:21,116] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:21,117] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_170358123/PGM_002_MR_1?format=json
[2020-02-18 17:07:21,125] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:21,126] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_170356545/PGM_002_MR_1?format=json
[2020-02-18 17:07:21,133] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:21,134] DEBUG:

In [34]:
# Get a list of the files in the prearchive that have READY status 
# and the session label experimentLabel:
session.prearchive.find(session=experimentLabel)

[2020-02-19 11:05:18,629] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects?format=json
[2020-02-19 11:05:18,658] DEBUG: session:307 >> Received response with status code: 200
[2020-02-19 11:05:18,660] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_163534273/PGM-002_Feb18_MR_1?format=json
[2020-02-19 11:05:18,674] DEBUG: session:307 >> Received response with status code: 200
[2020-02-19 11:05:18,675] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_170358123/PGM-002_Feb18_MR_1?format=json
[2020-02-19 11:05:18,694] DEBUG: session:307 >> Received response with status code: 200
[2020-02-19 11:05:18,695] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy/20200218_163533097/PGM-002_Feb18_MR_1?format=json
[2020-02-19 11:05:18,706] DEBUG: session:307 >> Received response with status code: 200
[2020-02-19 11:05:18,707] DEBUG: s

[<PrearchiveSession BrainTumorProg_copy/20200218_170358123/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170356545/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170354975/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170353407/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170353927/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170355982/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170355475/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170357605/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170357070/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170358634/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170359133/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/20200218_170359662/PGM-002_Feb18_MR_1>,
 <PrearchiveSession BrainTumorProg_copy/

In [25]:
len(prearchive_session_to_archive)

22

##### the project/subject/experiment don't need to be assigned when archiving since they were already assigned during prearchiving:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    archive_session = prearchive_session_to_archive[f].archive(project=projectLabel, \
                                                               subject=subjectLabel, \
                                                               experiment=experimentLabel)

### Rather than trying to move each file in series, see if I can move all of the at the same time:

In [27]:
#archive_session = prearchive_session_to_archive.archive(overwrite='append')
archive_session = prearchive_session_to_archive.archive()

AttributeError: 'list' object has no attribute 'archive'

In [32]:
# Only archive the desired files:

for f in range(len(prearchive_session_to_archive)):
    print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
    
    prearchive_session_to_archive[f].archive(project=projectLabel, \
                                             subject=subjectLabel, \
                                             experiment=experimentLabel, \
                                             overwrite='append')

[2020-02-18 17:07:29,953] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200218_170354463%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1
[2020-02-18 17:07:29,954] DEBUG: session:393 >> POST DATA None


Moving file 1 of 22 to the archive...


[2020-02-18 17:07:30,320] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:30,321] DEBUG: session:738 >> Type unknown, fetching data to get type
[2020-02-18 17:07:30,322] DEBUG: session:334 >> GET URI http://10.1.1.17/data/archive/projects/BrainTumorProg_copy/subjects/PGM-002_Feb18/experiments/PGM-002_Feb18_MR_1?format=json
[2020-02-18 17:07:30,407] DEBUG: session:307 >> Received response with status code: 200
[2020-02-18 17:07:30,407] DEBUG: session:746 >> Looking up type xnat:mrSessionData [str]
[2020-02-18 17:07:30,407] DEBUG: session:753 >> Creating object of type <class 'xnat_gen_ce9e3233f8742ed021c5bcfcb7b6320c.MrSessionData'>
[2020-02-18 17:07:30,408] DEBUG: session:391 >> POST URI http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200218_170358123%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1
[2020-02-18 17:07:30,408] DEBUG: session:393

Moving file 2 of 22 to the archive...


XNATResponseError: Invalid response from XNATSession for url http://10.1.1.17/data/services/archive?src=%2Fdata%2Fprearchive%2Fprojects%2FBrainTumorProg_copy%2F20200218_170358123%2FPGM_002_MR_1&overwrite=append&project=BrainTumorProg_copy&subject=PGM-002_Feb18&session=PGM-002_Feb18_MR_1 (status 409):
<html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Session processing may already be in progress: AutoRun.  Concurrent modification is discouraged.</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.4.10">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


# Current Warnings
## CONFLICT-14: Session already exists with matching files.
## CONFLICT-16: Session already contains a scan (11) with the same UID and number.

In [35]:
prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                        subject=subjectLabel, \
                                                        session=experimentLabel, \
                                                        status='READY')

[2020-02-19 11:25:23,224] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-19 11:25:23,249] DEBUG: session:307 >> Received response with status code: 200


In [36]:
len(prearchive_session_to_archive)

0

In [38]:
session.prearchive.find(project=projectLabel, \
                        subject=subjectLabel, \
                        session=experimentLabel)

[2020-02-19 11:26:19,277] DEBUG: session:334 >> GET URI http://10.1.1.17/data/prearchive/projects/BrainTumorProg_copy?format=json
[2020-02-19 11:26:19,300] DEBUG: session:307 >> Received response with status code: 200


[]

### Need to avoid trying to archive items that have already been archived:



status = 'True' # initial value

while status:
    # Update prearchive_session_to_archive:
    prearchive_session_to_archive = session.prearchive.find(project=projectLabel, \
                                                            subject=subjectLabel, \
                                                            session=experimentLabel, \
                                                            status='READY')
    
    #print('prearchive_session_to_archive = ', prearchive_session_to_archive)
    
    print('Number of files in prearchive_session_to_archive =', len(prearchive_session_to_archive), '\n')
    
    if len(prearchive_session_to_archive) == 0:
        status = 'False'
        break
        
    # Move each file in prearchive_session_to_archive to the archive:    
    for f in range(len(prearchive_session_to_archive)):
        print(f'Moving file {f+1} of {len(prearchive_session_to_archive)} to the archive...')
        
        prearchive_session_to_archive[f].archive(project=projectLabel, \
                                                 subject=subjectLabel, \
                                                 experiment=experimentLabel, \
                                                 overwrite='append')



### I posted another question to this 2018 post:

#### https://groups.google.com/forum/#!topic/xnat_discussion/JfR_CYasSrM

# I've not been able to upload more than one scan by importing the files individually.  Now try to upload an entire session in a single .zip file.

### I zipped all files into a single file "files.zip" and deleted all files within the prearchive that belong to subject PGM-002_Feb17.

In [42]:
#zippedDicomFpath = os.path.join(dicomDLdir, 'files.zip')
zippedDicomFpath = r'C:\Code\WP1.3_multiple_modalities\trying stuff\data\files.zip'
zippedDicomFpath

'C:\\Code\\WP1.3_multiple_modalities\\trying stuff\\data\\files.zip'

In [43]:
os.path.isfile(zippedDicomFpath)

True

### Last time I had run the following from the example shown in the XNATpy tutorial (which gave a Status 500 error):

In [99]:
prearchive_session = session.services.import_(zippedDicomFpath, \
                                              project=projectLabel, \
                                              subject=subjectLabel, \
                                              experiment=experimentLabel, \
                                              destination='/prearchive')

[2020-02-17 13:52:10,017] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy&subject=PGM-002_Feb17&session=PGM-002_Feb17_MR_1
[2020-02-17 13:52:10,073] DEBUG: session:307 >> Received response with status code: 500


XNATUploadError: Upload failed after 1 attempts! Status code 500, response text 

### In the example provided for the XNATpy tutorial, the importing is done with assigning only the project and destination. When moving from prearchive to archive the subject and experiment are assigned.  I tried that too:

In [103]:
prearchive_session = session.services.import_(zippedDicomFpath, \
                                              project=projectLabel, \
                                              destination='/prearchive')

[2020-02-17 14:04:36,838] DEBUG: session:661 >> UPLOAD URI http://10.1.1.17/data/services/import?dest=%2Fprearchive&project=BrainTumorProg_copy
[2020-02-17 14:04:36,881] DEBUG: session:307 >> Received response with status code: 500


XNATUploadError: Upload failed after 1 attempts! Status code 500, response text <html>
<head>
   <title>Status page</title>
</head>
<body>
<h3>Index: 0, Size: 0</h3><p>You can get technical details <a href="http://www.w3.org/Protocols/rfc2616/rfc2616-sec10.html#sec10.5.1">here</a>.<br>
Please continue your visit at our <a href="/">home page</a>.
</p>
</body>
</html>


### Feb 19:  Following the comment by Marcel Koek I'll try his suggestions:

##### Link: https://groups.google.com/forum/#!topic/xnat_discussion/JfR_CYasSrM

##### XNATpy is able to zip directories in place and upload them. Look at:

#####  1)  https://xnat.readthedocs.io/en/latest/xnat.html#xnat.mixin.AbstractResource.upload_dir
#####  2)  https://xnat.readthedocs.io/en/latest/static/tutorial.html#importing-data-into-xnat

##### In our experience it is indeed better and less messy to collect all data in the prearchive and trigger an archive action on the prearchive session:

#####  3)  https://xnat.readthedocs.io/en/latest/xnat.html#xnat.prearchive.PrearchiveSession.move
#####  4)  https://xnat.readthedocs.io/en/latest/xnat.html#xnat.prearchive.PrearchiveSession.archive

### I've already tried the .import_() function in the 2nd link.  So I'll try the .upload_dir() function in the 1st link.

### It's not clear how to use the functions but following the example on how to use .import_ in the first link:

### session.services.import_(full_filepath_to_zip_file, project='project_label', destination='/prearchive')

### and comparing that to the code reference here:

### https://xnat.readthedocs.io/en/latest/xnat.html#xnat.services.Services.import_

### class xnat.services.Services(xnat_session)

###  import_(path, overwrite=None, quarantine=False, destination=None, trigger_pipelines=None, project=None, subject=None, experiment=None, content_type=None)

### it looks like the second "Services" needs to be replaced with the functions shown below, e.g. import_().


### The code reference for the .upload_dir() function is:

###  class xnat.mixin.AbstractResource(uri=None, xnat_session=None, id_=None, datafields=None, parent=None, fieldname=None, overwrites=None, data_dir=None, upload_method=None, **kwargs)

###  upload_dir(directory, overwrite=False, method='tgz_file', **kwargs)

#####    Upload a directory to an XNAT resource. This means that if you do resource.upload_dir(directory) that if there is a file directory/a.txt it will be uploaded to resource/files/a.txt

#####    The method has 5 options, default is tgz_file:

#####        per_file: Scans the directory and uploads file by file
#####        tar_memory: Create a tar archive in memory and upload it in one go
#####        tgz_memory: Create a gzipped tar file in memory and upload that
#####        tar_file: Create a temporary tar file and upload that
#####        tgz_file: Create a temporary gzipped tar file and upload that

#####    The considerations are that sometimes you can fit things in memory so you can save disk IO by putting it in memory. The per file does not create additional archives, but has one request per file so might be slow when uploading many files.

#####    Parameters:	

#####        directory (str) – The directory to upload
#####        overwrite (bool) – Flag to force overwriting of files
#####        method (str) – The method to use

### Following the links found above for .upload_ it looks like the way to use it is:

### session.mixin.upload_dir(directory)

### which is different from:

### resource.upload_dir(directory) 

### in the code reference.  I'm not sure what "resource" is meant to be...

In [44]:
# First try:
session.mixin.upload_dir(dicomDLdir)

AttributeError: 'XNATSession' object has no attribute 'mixin'

In [45]:
# Second try:
resource.upload_dir(dicomDLdir)

NameError: name 'resource' is not defined

### I didn't think that would work, since resource is not defined.

In [46]:
# Third try:
session.upload_dir(dicomDLdir)

AttributeError: 'XNATSession' object has no attribute 'upload_dir'

In [48]:
# Since I'm trying to upload to a particular experiment maybe 
# that's what needs to be in place of resource:
experiment.upload_dir(dicomDLdir)

AttributeError: 'MrSessionData' object has no attribute 'upload_dir'

In [49]:
# Next try subject:
subject.upload_dir(dicomDLdir)

AttributeError: 'SubjectData' object has no attribute 'upload_dir'

In [50]:
# Maybe resource needs to be added to experiment?
experiment.resource.upload_dir(dicomDLdir)

AttributeError: 'MrSessionData' object has no attribute 'resource'

### Maybe I need to create an assessor for experiment, and then a resource for the assessor?...